In [184]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
from PIL import Image

plt.style.use('dark_background')

In [185]:
server = '1716'

In [186]:
cnt = 1

for i in range(50):
    test = Image.open('./data/' + server + '/original/' + str(i+1) + '.png')
    x1, x2 = 1400, 1600

    test_crop = test.crop((x1, 300, x2, 390))
    test_crop1 = test.crop((x1, 430, x2, 520))
    test_crop2 = test.crop((x1, 550, x2, 640))
    test_crop3 = test.crop((x1, 670, x2, 770))
    test_crop4 = test.crop((x1, 790, x2, 890))
    test_crop5 = test.crop((x1, 900, x2, 1000))

    test_crop.save('./data/' + server + '/power/' + str(cnt) + '.png')
    cnt += 1
    test_crop1.save('./data/' + server + '/power/' + str(cnt) + '.png')
    cnt += 1
    test_crop2.save('./data/' + server + '/power/' + str(cnt) + '.png')
    cnt += 1
    test_crop3.save('./data/' + server + '/power/' + str(cnt) + '.png')
    cnt += 1
    test_crop4.save('./data/' + server + '/power/' + str(cnt) + '.png')
    cnt += 1
    test_crop5.save('./data/' + server + '/power/' + str(cnt) + '.png')
    cnt += 1

In [187]:
power = list()
for i in range(300):
    img_ori = cv2.imread('data/' + server + '/power/' + str(i+1) + '.png')

    gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
        img_blurred, 
        127,
        255,
        cv2.THRESH_BINARY_INV
    )

    chars = pytesseract.image_to_string(img_thresh)
    list_of_numbers = re.findall(r'\d+', chars)
    result_number = ''.join(list_of_numbers)

    if result_number == '':
        result_number = '0'

    power.append(int(result_number))

In [188]:
result = pd.DataFrame({'POWER': power})

In [192]:
pd.options.display.float_format = '{:.5f}'.format
describe = result.describe()
describe

,POWER
count,300.00000
mean,25860905.92667
std,12955217.76179
min,14656652.00000
25%,18099240.75000
50%,21901716.00000
75%,28319949.75000
max,110476763.00000


In [190]:
result.to_excel('./data/' + server + '/result_v1.xlsx')
describe.to_excel('./data/' + server + '/describe_v1.xlsx')